# Week 11 Exercises (13-1)

## Exercise 13-1
In NSFG Cycles 6 and 7, the variable cmdivorcx contains the date of divorce for the
respondent’s first marriage, if applicable, encoded in century-months.

Compute the duration of marriages that have ended in divorce, and the duration, so far,
of marriages that are ongoing. Estimate the hazard and survival function for the duration
of marriage.

Use resampling to take into account sampling weights, and plot data from several resamples
to visualize sampling error.

Consider dividing the respondents into groups by decade of birth, and possibly by age
at first marriage.

In [3]:
import nsfg
import thinkstats2
import survival
import numpy as np

In [6]:
preg = nsfg.ReadFemPreg()
resp = nsfg.ReadFemResp()
# clean divorce and marriage variables
resp.cmdivorcx.replace([9997, 9998, 9999], np.nan, inplace=True)
resp.cmmarrhx.replace([9997, 9998, 9999], np.nan, inplace=True)

# creates variables for those not divorced
resp['notdivorced'] = resp.cmdivorcx.isnull().astype(int)
# duration of marriages that ended in divorce
resp['duration'] = (resp.cmdivorcx - resp.cmmarrhx) / 12.0
#computes duration of those still married
resp['duration_now'] = (resp.cmintvw - resp.cmmarrhx) / 12.0

In [20]:
def EstimateDivorce(resp):
    """calculates hazard and survival function 
    for divorces
    """
    # variables for those divorced and those still married
    complete = resp[resp.notdivorced== 0].duration.dropna()
    ongoing = resp[resp.notdivorced == 1].duration_now.dropna()
    
    # hazard function for duration of marriage
    hazard = survival.EstimateHazardFunction(complete, ongoing)
    # survival function for duration of marriage
    surv = hazard.MakeSurvival(hazard)

In [19]:
def ResampleDivorce(resp, iters = 101):
    """Resamples respondents and computes survival function 
    to account for sampling weights"""
    for i in range(iters):
        samples = thinkstats2.ResampleRowsWeighted(resp)
        hf, sf = EstimateDivoce(samples)
        sample = pd.concat(samples, ignore_index=True)
    thinkplot.Config(xlabel = "years", ylabel = "Fraction undivorced", axis = [0, 28, 0, 1])